In [1]:
import pandas as pd 
import numpy as np
import sys

In [2]:
fact_df = pd.read_csv('dimensions/fact_table_mob.csv')
date_dimension = pd.read_csv('dimensions/date_dimension.csv')
weather_dimension = pd.read_csv('dimensions/weather_dimension.csv')
phu_dimension = pd.read_csv('dimensions/PHU_location_dimension.csv')

In [3]:
date_dimension.dtypes

date_key                  int64
date                     object
date_id                   int64
day_name                 object
day_name_abbrev          object
day_of_month              int64
day_of_week               int64
day_of_year               int64
holiday_name             object
is_holiday                 bool
is_weekday                 bool
is_weekend                 bool
month_abbrev             object
month_end_flag             bool
month_name               object
month_number              int64
quarter                   int64
quarter_name             object
quarter_short_name       object
season                   object
week_begin_date          object
week_begin_date_id        int64
week_num_in_month         int64
week_num_in_year          int64
year                      int64
year_and_month           object
year_and_month_abbrev    object
year_and_quarter         object
dtype: object

In [4]:
# Convert dates to date objects
date_dimension['date'] = pd.to_datetime(date_dimension['date'])
date_dimension['week_begin_date'] = pd.to_datetime(date_dimension['week_begin_date'])

In [5]:
weather_dimension['date'] = pd.to_datetime(weather_dimension['date'])

In [6]:
weather_dimension.tail()

,weather_Key,city,date,daily_low_temperature,daily_high_temperature,precipitation
243,243,Ottawa,2021-02-02,-10.2,0.6,1.8
244,244,Ottawa,2021-02-03,-4.3,1.2,2.0
245,245,Ottawa,2021-02-04,-9.8,0.7,0.0
246,246,Ottawa,2021-02-05,-9.7,1.8,1.8
247,247,Ottawa,2021-02-06,-12.2,-4.7,0.0


In [7]:
weather_dimension.city.unique()

array(['Toronto', 'Ottawa'], dtype=object)

In [8]:
fact_df.head()

,Onset_date_ID,Reported_date_ID,Test_date_ID,Specimen_date_ID,Patient_ID,PHU_Location_ID,Mobility_ID,Special_Measures_ID,Weather_ID,resolved,unresolved,fatal
0,20200311,20201205,20201205,20200929,1,2253,432,5,-1,yes,no,no
1,20200323,20201212,20201212,20201211,2,2270,563,3,-1,yes,no,no
2,20200429,20201006,20201006,20200821,3,2270,496,14,-1,yes,no,no
3,20201002,20201006,20201006,20201002,6,2230,124,14,-1,yes,no,no
4,20200926,20201006,20201006,20200930,7,2270,496,14,-1,yes,no,no


In [9]:
phu_dimension.head()

,PHU_location_key,PHU_id,PHU_name,Address,City,Postal_code,Province,URL,Latitude,Longitude
0,0,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,Ontario,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,1,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,Ontario,www.peelregion.ca/health/,43.647471,-79.708893
2,2,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,Ontario,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
3,3,2236,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,Ontario,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796
4,4,2230,Durham Region Health Department,605 Rossland Road East,Whitby,L1N 0B2,Ontario,www.durham.ca/en/health-and-wellness/health-an...,43.898605,-78.940341


In [10]:
phu_dimension['PHU_name'].unique()

array(['Toronto Public Health', 'Peel Public Health',
       'York Region Public Health Services',
       'Halton Region Health Department',
       'Durham Region Health Department', 'Ottawa Public Health'],
      dtype=object)

In [11]:
phu_dimension.head()

,PHU_location_key,PHU_id,PHU_name,Address,City,Postal_code,Province,URL,Latitude,Longitude
0,0,3895,Toronto Public Health,"277 Victoria Street, 5th Floor",Toronto,M5B 1W2,Ontario,www.toronto.ca/community-people/health-wellnes...,43.656591,-79.379358
1,1,2253,Peel Public Health,7120 Hurontario Street,Mississauga,L5W 1N4,Ontario,www.peelregion.ca/health/,43.647471,-79.708893
2,2,2270,York Region Public Health Services,17250 Yonge Street,Newmarket,L3Y 6Z1,Ontario,www.york.ca/wps/portal/yorkhome/health/,44.048023,-79.480239
3,3,2236,Halton Region Health Department,1151 Bronte Road,Oakville,L6M 3Ll,Ontario,www.halton.ca/For-Residents/Public-Health/,43.413997,-79.744796
4,4,2230,Durham Region Health Department,605 Rossland Road East,Whitby,L1N 0B2,Ontario,www.durham.ca/en/health-and-wellness/health-an...,43.898605,-78.940341


In [12]:
phu_subregion_map = {
    'Ottawa Public Health' : 'Ottawa',
    'Durham Region Health Department' : 'Toronto',
    'Halton Region Health Department' : 'Toronto',
    'Peel Public Health' : 'Toronto',
    'York Region Public Health Services' : 'Toronto',
    'Toronto Public Health' : 'Toronto'
}

In [13]:
print(weather_dimension.columns)

Index(['weather_Key', 'city', 'date', 'daily_low_temperature',
       'daily_high_temperature', 'precipitation'],
      dtype='object')


In [14]:
fact_table_mob_mapped = pd.DataFrame(columns=fact_df.columns)

In [15]:
len(fact_df)

148053

In [16]:
y = 112352

for idx, row in fact_df.iterrows():
        
    sys.stdout.write("\r{0} out of {1} generated...".format(idx,y))
    fact_table_row = []
    fact_table_row.append(row['Onset_date_ID'])
    fact_table_row.append(row['Reported_date_ID'])
    fact_table_row.append(row['Test_date_ID'])
    fact_table_row.append(row['Specimen_date_ID'])
    fact_table_row.append(row['Patient_ID'])
    fact_table_row.append(row['PHU_Location_ID'])
    fact_table_row.append(row['Mobility_ID'])
    fact_table_row.append(row['Special_Measures_ID'])
    
    # Grab the PHU name 
    phu_name = phu_dimension[phu_dimension['PHU_id']==row['PHU_Location_ID']]['PHU_name'].values[0]
    reported_date = date_dimension[date_dimension['date_id']==row['Reported_date_ID']]['date'].values[0]
    
    weather_Key = weather_dimension[(weather_dimension['date'] == reported_date) &
            (phu_subregion_map[phu_name] == weather_dimension["city"])]['weather_Key'].values[0]
    
    fact_table_row.append(weather_Key)
    fact_table_row.append(row['resolved'])
    fact_table_row.append(row['unresolved'])
    fact_table_row.append(row['fatal'])
    
    fact_table_mob_mapped.loc[len(fact_table_mob_mapped)] = fact_table_row
 

148052 out of 112352 generated...

In [17]:
fact_table_mob_mapped.to_csv('dimensions/fact_table_mob_weather.csv', index=False)

In [18]:
fact_table_mob_mapped.head()

,Onset_date_ID,Reported_date_ID,Test_date_ID,Specimen_date_ID,Patient_ID,PHU_Location_ID,Mobility_ID,Special_Measures_ID,Weather_ID,resolved,unresolved,fatal
0,20200311,20201205,20201205,20200929,1,2253,432,5,60,yes,no,no
1,20200323,20201212,20201212,20201211,2,2270,563,3,67,yes,no,no
2,20200429,20201006,20201006,20200821,3,2270,496,14,0,yes,no,no
3,20201002,20201006,20201006,20201002,6,2230,124,14,0,yes,no,no
4,20200926,20201006,20201006,20200930,7,2270,496,14,0,yes,no,no


In [19]:
set(fact_table_mob_mapped.Weather_ID)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,


In [20]:
weather_dimension_nodate = weather_dimension.drop(columns='date', inplace=True)

In [21]:
weather_dimension_nodate.head()

AttributeError: 'NoneType' object has no attribute 'head'

In [22]:
weather_dimension.to_csv('dimensions/weather_dimension_nodate.csv', index=False)